In [4]:
%%file layers.py
# %load layers.py
from inits import *
import tensorflow as tf

flags = tf.app.flags
FLAGS = flags.FLAGS

# global unique layer ID dictionary for layer name assignment
_LAYER_UIDS = {}


def get_layer_uid(layer_name=''):
    """Helper function, assigns unique layer IDs."""
    if layer_name not in _LAYER_UIDS:
        _LAYER_UIDS[layer_name] = 1
        return 1
    else:
        _LAYER_UIDS[layer_name] += 1
        return _LAYER_UIDS[layer_name]

def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res    
    
class Layer(object):
    """Base layer class. Defines basic API for all layer objects.
    Implementation inspired by keras (http://keras.io).

    # Properties
        name: String, defines the variable scope of the layer.
        logging: Boolean, switches Tensorflow histogram logging on/off

    # Methods
        _call(inputs): Defines computation graph of layer
            (i.e. takes input, returns output)
        __call__(inputs): Wrapper for _call()
        _log_vars(): Log all variables
    """

    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            layer = self.__class__.__name__.lower()
            name = layer + '_' + str(get_layer_uid(layer))
        self.name = name
        self.vars = {}
        logging = kwargs.get('logging', False)
        self.logging = logging

    def _call(self, inputs):
        return inputs

    def __call__(self, inputs):
        with tf.name_scope(self.name):
            if self.logging:
                tf.summary.histogram(self.name + '/inputs', inputs)
            outputs = self._call(inputs)
            if self.logging:
                tf.summary.histogram(self.name + '/outputs', outputs)
            return outputs

    def _log_vars(self):
        for var in self.vars:
            tf.summary.histogram(self.name + '/vars/' + var, self.vars[var])

class Flatten(Layer):
    """Flattens a tensor layer."""
    def __init__(self, **kwargs):
        super(Flatten, self).__init__(**kwargs)

    def _call(self, inputs):
        x = inputs
        # flatten the tensor to one layer
        shape = x.get_shape().as_list()               # a list: [None,...]
        dim = np.prod(shape[1:])                   # dim = prod(...)
        x_flattened = tf.reshape(x, [-1, dim])        # -1 means "all"
        return x_flattened
    
class Dense(Layer):
    """Dense layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 act=tf.nn.relu, bias=False, featureless=False, **kwargs):
        super(Dense, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.featureless = featureless
        self.bias = bias

        with tf.variable_scope(self.name + '_vars'):
            self.vars['weights'] = glorot([input_dim, output_dim],
                                          name='weights')
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')

        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs

        # dropout
        x = tf.nn.dropout(x, 1-self.dropout)
        
        # add dummy dimension
        x = tf.expand_dims(x, 1) # Batch1M
        
        # transform
        (batch, n, m) = x.get_shape().as_list()
        p = self.vars['weights'].get_shape().as_list()[1]
        output = tf.reshape(tf.reshape(x, [-1, m]) @ self.vars['weights'], [-1, n, p]) # BatchNM * MP => Batch1P (N should be 1)
        
        # bias
        if self.bias:
            output += self.vars['bias']

        return self.act(output) # Batch1P

class GraphConvolution(Layer):
    """Graph convolution layer."""
    def __init__(self, input_dim, output_dim, placeholders, dropout=0.,
                 act=tf.nn.relu, bias=False,
                **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)

        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.

        self.act = act
        self.support = placeholders['support']
        self.bias = bias
        self.num_nodes = self.support.get_shape().as_list()[2]
        self.output_dim = output_dim
        
        with tf.variable_scope(self.name + '_vars'):
            # make all weight matrices for supports in convolution
            for i in range(self.support.get_shape().as_list()[1]):# support: ?xSupportsxNxNxM
                for j in range(self.support.get_shape().as_list()[4]):
                    tensor_name = 'weights_support_' + str(i) + '_M_' + str(j)
                    self.vars[tensor_name] = glorot([input_dim, output_dim], name=tensor_name)
            # make vector to do weighted sum of all convolved features (w in SUM(wi*(NxF')) for w in M)
            self.vars["Features Combination"] = uniform(self.support.get_shape().as_list()[4],
                                                            name="Features Combination")
            # make bias matrice
            if self.bias:
                self.vars['bias'] = zeros([output_dim], name='bias')
            
        
        if self.logging:
            self._log_vars()

    def _call(self, inputs):
        x = inputs
        
        # dropout
        x = tf.nn.dropout(x, 1-self.dropout)
        
        # convolve
        convolved_features = []
        for j in range(self.support.get_shape().as_list()[4]):
            temp = []
            for i in range(self.support.get_shape().as_list()[1]):
                tensor_name = 'weights_support_' + str(i) + '_M_' + str(j)
                # BatchNF * FF' weight tensor (num_nodes, |Features'|)
                (N, F) = x.get_shape().as_list()[1:]
                embed = tf.reshape(x, [-1, F])
                pre_sup =  tf.reshape(tf.reshape(x, [-1, F]) @ self.vars[tensor_name], [-1, N, self.output_dim])
                (batch, _, F_new) = pre_sup.get_shape().as_list()

                # BatchNN * BatchNF' => BatchNF'
                support = tf.slice(self.support, [0,i,0,0,j], [-1,1,-1,-1,1]) # get Batch1NN1
                support = tf.reshape(support, [-1,N,N]) # reshape to BatchNN
                support = support @ pre_sup # now BatchNF'
                temp.append(support)
            # adds together list of BatchNF' into one BatchNF' for a single original adjacency matrix
            convolved_F = tf.add_n(temp)
            convolved_features.append(convolved_F)
        # stack list into one tensor of shape BatchNF'M
        convolved_features = tf.stack[convolved_features, axis = 3]
        # do weighted multiplication
        tf.multiply(convolved_features, self.vars["Features Combination"])
        # sum together to remove 4th dimension
        output = tf.add_n(convolved_features, axis = 3)
        
        # bias
        if self.bias:
            output += self.vars['bias'] # Broadcasting spreads bias across Batch and Node dimensions

        return self.act(output)

class SelfAttention(Layer):
    """Self attention layer, input is in ?xNxhidden, output is in ?x(Bias*Hidden). Hidden should correspond 
    to the number of features nodes have."""
    
    def __init__(self, attention_dim, bias_dim, hidden_units, placeholders, dropout=0., **kwargs):
        super().__init__(**kwargs)
        
        if dropout:
            self.dropout = placeholders['dropout']
        else:
            self.dropout = 0.
        
        self.hidden_units = hidden_units
        self.A = None
        self.vars['Ws'] = tf.Variable(tf.random_uniform([attention_dim, self.hidden_units])) # AttentionxHidden
        self.vars['W2'] = tf.Variable(tf.random_uniform([bias_dim, attention_dim])) # BiasxAttention

    def _call(self, inputs):
        
        # dropout
        inputs = tf.nn.dropout(inputs, 1-self.dropout)
        
        # AttentionxHidden * ?xHiddenxN => ?xAttentionxN
        inputsT = tf.transpose(inputs, perm = [0, 2, 1]) # transpose the inner matrices which is our intention
        
        #print("inputsT is of shape {}".format(inputsT.get_shape().as_list()))
        #print("self.vars[Ws] is of shape {}".format(self.vars['Ws'].get_shape().as_list()))
        
        aux = tf.einsum('ah,bhn->ban', self.vars['Ws'], inputsT)
        aux = tf.tanh(aux)
        
        #print("aux is of shape {}".format(aux.get_shape().as_list()))
        
        
        # BiasxAttention * ?xAttentionxN => ?xBiasxN
        self.A = tf.einsum('ba,uan->ubn',self.vars['W2'], aux)
        self.A = tf.nn.softmax(self.A)
        #print("A is of shape {}".format(self.A.get_shape().as_list()))
        #tf.summary.histogram('self_attention', self.A) For visualization
        
        # ?xBiasxN * ?xNxHidden => ?xBiasxHidden
        out = self.A @ inputs
        #print("out is of shape {}".format(out.get_shape().as_list()))
        
        # ?xBiasxHidden => ?x(Bias*Hidden)
        out = tf.reshape(out, [ -1, out.get_shape().as_list()[1] * out.get_shape().as_list()[2]])
        #print("out is of shape {}".format(out.get_shape().as_list()))
        return out

Overwriting layers.py


In [5]:
%%file models.py
# %load models.py
from layers import *
from metrics import *

flags = tf.app.flags
FLAGS = flags.FLAGS

class Model(object):
    def __init__(self, **kwargs):
        allowed_kwargs = {'name', 'logging'}
        for kwarg in kwargs.keys():
            assert kwarg in allowed_kwargs, 'Invalid keyword argument: ' + kwarg
        name = kwargs.get('name')
        if not name:
            name = self.__class__.__name__.lower()
        self.name = name

        logging = kwargs.get('logging', False)
        self.logging = logging

        self.vars = {}
        self.placeholders = {}

        self.layers = []
        self.activations = []

        self.inputs = None
        self.outputs = None
        self.logits = None
        self.predictions = None
        
        self.loss = 0
        self.accuracy = 0
        self.optimizer = None
        self.opt_op = None

    def _build(self):
        raise NotImplementedError

    def build(self):
        """ Wrapper for _build() """
        with tf.variable_scope(self.name):
            self._build()

        # Build sequential layer model
        self.activations.append(self.inputs)
        for layer in self.layers:
            hidden = layer(self.activations[-1])
            self.activations.append(hidden)
        #print("\n\n\nactivations\n{}\n\n\n".format(self.activations))
        self.outputs = self.activations[-1]

        # Store model variables for easy access
        variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self.name)
        self.vars = {var.name: var for var in variables}

        # Build metrics
        self._loss()
        self._accuracy()

        self.opt_op = self.optimizer.minimize(self.loss)

    def predict(self):
        pass

    def _loss(self):
        raise NotImplementedError

    def _accuracy(self):
        raise NotImplementedError

    def save(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = saver.save(sess, "tmp/%s.ckpt" % self.name)
        print("Model saved in file: %s" % save_path)

    def load(self, sess=None):
        if not sess:
            raise AttributeError("TensorFlow session not provided.")
        saver = tf.train.Saver(self.vars)
        save_path = "tmp/%s.ckpt" % self.name
        saver.restore(sess, save_path)
        print("Model restored from file: %s" % save_path)


class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)
        
        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders
        self.number_nodes = placeholders['features'].get_shape().as_list()[1]
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)
        
        #print("placeholders and descriptions:")
        #for key in self.placeholders:
        #    print("{}: {}".format(key, self.placeholders[key]))
        
        #print("Model's input {}, output {}".format(self.input_dim, self.output_dim))
        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)
        
        # Cross entropy error
        labels = self.placeholders['labels']
        logits = self.outputs
        dim = logits.get_shape().as_list()[2]
        logits = tf.reshape(logits, [-1, dim])
        entropies = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = labels, dim = -1)
        loss = tf.reduce_mean(entropies)
        #print("Labels {} \nLogits {}\nEntropies {}\nLoss {}\n\n".format(labels, logits, entropies, loss))
        self.loss += loss

    def _accuracy(self):
        labels = self.placeholders['labels']
        logits = self.outputs
        dim = logits.get_shape().as_list()[2]
        logits = tf.reshape(logits, [-1, dim])
        self.logits = logits
        labels=tf.argmax(labels, 1) # labels
        predictions=tf.argmax(logits, 1) # prediction as one hot
        self.predictions = predictions
        
        # Define the metric and update operations
        tf_metric, tf_metric_update = tf.metrics.accuracy(predictions = predictions, labels = labels, name = "accuracy")
        self.accuracy = tf_metric_update
        
        # Isolate the variables stored behind the scenes by the metric operation
        running_vars = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="accuracy")
        # Define initializer to initialize/reset running variables
        self.running_vars_initializer = tf.variables_initializer(var_list=running_vars)

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            logging=self.logging))
        
        #print("first layer: input {}, output {}".format(self.input_dim, FLAGS.hidden1))
        
        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=FLAGS.hidden2,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            logging=self.logging))
    
        #print("second layer: input {}, output {}".format(FLAGS.hidden1, FLAGS.hidden2))
        
        ####### flatten the output so that it is now one continuous layer
        #####self.layers.append(Flatten(logging=self.logging))
        
        # perform self attention, hidden must be number of features, or hidden2. Other two are more arbitrary
        self.layers.append(SelfAttention(attention_dim = FLAGS.attention_dim,
                                         bias_dim = FLAGS.attention_bias, 
                                         hidden_units = FLAGS.hidden2,
                                         placeholders=self.placeholders,
                                         dropout=True,
                                         logging = self.logging))
        
        #print("self attention with attention_dim {}, bias_dim {}:\n input {}, output(bias*input) {}".format(FLAGS.attention_dim, FLAGS.attention_bias, [self.number_nodes, FLAGS.hidden2] ,FLAGS.hidden2 * FLAGS.attention_bias))
        
        # dense FC layer to get out an output prediction
        self.layers.append(Dense(input_dim=FLAGS.hidden2 * FLAGS.attention_bias,
                                 output_dim=self.output_dim,
                                 act=lambda x: x,
                                 placeholders=self.placeholders,
                                 dropout=True,
                                 logging=self.logging))
        
        #print("final layer: input {}, output {}".format(FLAGS.hidden2 * FLAGS.attention_bias, self.output_dim))
        
    def predict(self):
        logits = self.outputs
        dim = logits.get_shape().as_list()[2]
        logits = tf.reshape(logits, [-1, dim])
        return tf.nn.softmax(logits)


Overwriting models.py


In [6]:
%%file utils.py
# %load utils.py
import numpy as np
import pickle as pkl
import networkx as nx
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
import scipy.sparse as sp
import tensorflow as tf
import os

def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def load_data(dataset_str):
    """
    Loads input data from gcn/data directory

    ind.dataset_str.x => arr of the feature vectors of the training instances as numpy.ndarray object;
    ind.dataset_str.y => arr of the one-hot labels of the labeled training instances as numpy.ndarray object (|label| = number of classes); 
    ind.dataset_str.graph => arr of adjacency matrices as numpy objects
    ind.dataset_str.test.index => index file for test values. To ensure we properly do ONE split for all possible hyperparameters
    it simply is in Data/ind.all.test.index. This is NOT regenerated

    All objects above must be saved using python pickle module.

    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """
    cwd = os.getcwd()
    os.chdir("..")
    names = ['x', 'y', 'graph', 'sequences', 'labelorder']
    objects = []
    for i in range(len(names)):
        with open("Data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x_arr, y_arr, graph_arr, sequences, labelorder = tuple(objects)
    
    # this is the tensor of datapoints converted to a list of sparse matrices (BATCHxNxF)
    features = x_arr
    
    # make all the adjacency lists into nx graph objects (BATCHxGRAPHS)
    adj_ls = graph_arr
    
    # read in the test indices from the index file
    test_idx_reorder = parse_index_file("Data/ind.all.test.index")
    test_idx_range = np.sort(test_idx_reorder)
    idx_test = test_idx_range.tolist()
    
    os.chdir(cwd)
    
    # get training indexes and then split this group up into testing and validation
    idx_train = [y_ind for y_ind in range(y_arr.shape[0]) if y_ind not in idx_test]
    np.random.shuffle(idx_train)
    cutoff = int(6*len(idx_train)/7)
    idx_val = idx_train[cutoff:]
    idx_train = idx_train[:cutoff]
    idx_train, idx_val = np.sort(idx_train), np.sort(idx_val)
    idx_test = np.array(idx_test)
    
    # make logical indices (they are the size BATCH)
    train_mask = sample_mask(idx_train, y_arr.shape[0])
    val_mask = sample_mask(idx_val, y_arr.shape[0])
    test_mask = sample_mask(idx_test, y_arr.shape[0])
    
    
    
    print("|Training| {}, |Validation| {}, |Testing| {}".format(len(idx_train), len(idx_val), len(idx_test)))
    
    return adj_ls, features, y_arr, sequences, labelorder, train_mask, val_mask, test_mask


def preprocess_features(features):
    """Row-normalize feature matrix and convert to tuple representation"""
    for i in range(features.shape[0]):
        feature_arr = features[i,:,:]
        rowsum = np.array(feature_arr.sum(1))
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = np.diag(r_inv)
        features[i,:,:] = r_mat_inv.dot(feature_arr)
    return features


def normalize_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    # added a shift to be non negative
    adj += np.amax(adj)
    # normalize
    rowsum = adj.sum(1)
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = np.diag(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt)


def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + np.identity(adj.shape[0]))
    return adj_normalized



def construct_feed_dict(features, support, labels, placeholders):
    """Construct feed dictionary."""
    feed_dict = dict()
    feed_dict.update({placeholders['labels']: labels})
    feed_dict.update({placeholders['features']: features})
    feed_dict.update({placeholders['support']: support})
    return feed_dict


def chebyshev_polynomials(adj, k):
    """Calculate Chebyshev polynomials up to order k. Return a list of sparse matrices (tuple representation)."""

    adj_normalized = normalize_adj(adj)
    laplacian = np.identity(adj.shape[0]) - adj_normalized
    largest_eigval, _ = eigsh(laplacian, 1, which='LM') # should still work
    scaled_laplacian = (2. / largest_eigval[0]) * laplacian - np.identity(adj.shape[0])

    t_k = list()
    t_k.append(np.identity(adj.shape[0]))
    t_k.append(scaled_laplacian)

    def chebyshev_recurrence(t_k_minus_one, t_k_minus_two, scaled_lap):
        s_lap = sp.csr_matrix(scaled_lap, copy=True)
        return 2 * s_lap.dot(t_k_minus_one) - t_k_minus_two

    for i in range(2, k+1):
        t_k.append(chebyshev_recurrence(t_k[-1], t_k[-2], scaled_laplacian))

    return t_k


Overwriting utils.py


In [7]:
%%file train.py
# %load train.py
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf
import numpy as np
from utils import *
from models import GCN
import pandas as pd
import os

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('dataset', 'testset', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1 (graph conv layer).')
flags.DEFINE_integer('hidden2', 7, 'Number of units in hidden layer 2 (graph conv layer).')
flags.DEFINE_integer('attention_bias', 2, 'Attention Bias.')
flags.DEFINE_integer('attention_dim', 5, 'Attention Dimension.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')
flags.DEFINE_string('save_validation', "False", "If you should save validation accuracy")
flags.DEFINE_string('save_test', "False", "If this is optimized run! Use all data and save outputs")


# Load data
adj_ls, features, y_arr, sequences, labelorder, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

#print("Train {}".format(train_mask))
#print("Val {}".format(val_mask))
#print("Test {}\n".format(test_mask))

# Save with a name defined by model params
model_desc = "hidden1_{0}_hidden2_{1}_dropout_{2}_attdim_{3}_attbias_{4}_model_{5}_maxdeg_{6}"
model_desc = model_desc.format(FLAGS.hidden1, FLAGS.hidden2, FLAGS.dropout, FLAGS.attention_dim,
                              FLAGS.attention_bias, FLAGS.model, FLAGS.max_degree)

# determine num_supports and make model function
if FLAGS.model == 'gcn':
    num_supports = 1
    model_func = GCN
elif FLAGS.model == 'gcn_cheby':
    num_supports = 1 + FLAGS.max_degree
    model_func = GCN
else:
    raise ValueError('Invalid argument for model: ' + str(FLAGS.model))

# save validation
save_validation = FLAGS.save_validation
if save_validation == "True":
    save_validation = True
else:
    save_validation = False

# see if we are saving the output by considering testing set
save_test = FLAGS.save_test
if save_test == "True":
    save_test = True
else:
    save_test = False

if save_test:
    epoch_df = pd.DataFrame(np.zeros(shape = (FLAGS.epochs, 5)))
    epoch_df.columns = ["train_loss", "train_acc", "val_acc", "val_loss", "time"]
    labels_df = pd.DataFrame(np.zeros(shape = (sum(test_mask), 5)))
    labels_df.columns = ["Sequence", "Label", "Prediction", "Negative Class Logit", "Positive Class Logit"]
    

# preload support tensor so that it isn't needlessly calculated many times
batch,_,N,M = adj_ls.shape
support_tensor = np.zeros(shape=(batch,num_supports,N,N,M)) # of shape (Batch,Num_Supports,Num_Nodes,Num_Nodes)
print("Calculating Chebyshev polynomials up to order {}...".format(FLAGS.max_degree))
for b in range(batch):
    adj = adj_ls[b]
    for m in range(M):
        adj = adj_ls[b][:,:,m] # first adjacency list
        if FLAGS.model == 'gcn':
            support = [preprocess_adj(adj)]
        elif FLAGS.model == 'gcn_cheby':
            support = chebyshev_polynomials(adj, FLAGS.max_degree)
        # add NxN matrices along the num_supports dimension
        sup = np.stack(support, axis=0)
        # add num_supportsxNxN to support tensor
        support_tensor[b,:,:,:,m] = sup

# normalize all features
features = preprocess_features(features)

# Define placeholders
F = features.shape[2]
placeholders = {
    'support': tf.placeholder(tf.float32, shape=(None,num_supports,N,N,M)), # ?xnum_supportsxNxNxM
    'features': tf.placeholder(tf.float32, shape=(None,N,F)), # ?xNxF
    'labels': tf.placeholder(tf.float32, shape=(None, y_arr.shape[1])), # ?,|labels|
    'dropout': tf.placeholder_with_default(0., shape=()),
}

# Create model
model = model_func(placeholders, input_dim=features.shape[2], logging=True)

# Initialize session
sess = tf.Session()

# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders, model):
    t_test = time.time()
    features = features[mask,:,:]
    support = support[mask,:,:,:]
    labels = labels[mask, :]
    feed_dict_val = construct_feed_dict(features, support, labels, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)

# Init variables
sess.run(tf.global_variables_initializer())
sess.run(model.running_vars_initializer)

# Train model
ttot = time.time()
cost_val = []
acc_val = []
for epoch in range(FLAGS.epochs):
    t = time.time()

    # instantiate all inputs
    features_train = features[test_mask,:,:]
    support = support_tensor[test_mask,:,:,:]
    y_test = y_arr[test_mask, :]

    # Construct feed dictionary
    feed_dict = construct_feed_dict(features_train, support, y_test, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Reset the counters
    sess.run(model.running_vars_initializer)
    
    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Reset the counters
    sess.run(model.running_vars_initializer)
    
    # Validation
    cost, acc, duration = evaluate(features, support_tensor, y_arr, val_mask, placeholders, model)

    cost_val.append(cost)
    
    # save training progression
    if save_test:
        epoch_df.iloc[epoch, :] = [outs[1], outs[2], cost, acc, time.time() - t]
    
    # Print results
    if (epoch + 1) % 20 == 0:
        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping + 1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished! Total Time: {} sec".format(time.time() - ttot))

# Reset the counters
sess.run(model.running_vars_initializer)

# Testing
test_cost, test_acc, test_duration = evaluate(features, support_tensor, y_arr, test_mask, placeholders, model)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

# save a text file whose name is the model_desc, graph_desc (from dataset), and has this info within it  
if save_validation:
    root = os.getcwd()
    optimization = os.path.join(root, "Optimization")
    validation_accuracy = acc
    filename = "{0:.5f}.{1}.{2}.outcome.txt".format(validation_accuracy, model_desc, FLAGS.dataset)
    filename = os.path.join(optimization, filename)
    with open(filename, "w") as fh:
        fh.write("{}\n{}\n{}".format(validation_accuracy, model_desc, FLAGS.dataset))
    

# Saving results to a file
if save_test:
    # get test values
    features_test = features[test_mask,:,:]
    support_test = support_tensor[test_mask,:,:,:]
    labels_test = y_arr[test_mask, :]
    
    # add sequences
    labels_df.iloc[:, 0] = [sequences[i] for i in range(len(test_mask)) if test_mask[i]]
    
    # add true labels
    labels_df.iloc[:, 1] = [np.where(labels_test[i])[0] for i in range((sum(test_mask)))]
    
    # get logits in final layer
    feed_dict_val = construct_feed_dict(features_test, support_test, labels_test, placeholders)
    logits, predictions = sess.run([model.logits, model.predictions], feed_dict=feed_dict_val)
    labels_df.iloc[:, 3:5] = logits
    
    # get predictions
    labels_df.iloc[:, 2] = predictions
    
    # change indices for labels to their names
    labels_df.iloc[:,1] = labels_df.iloc[:,1].map(lambda x: labelorder[x])
    labels_df.iloc[:,2] = labels_df.iloc[:,2].map(lambda x: labelorder[x])
    
    # write to file
    epoch_df.to_csv("../Results/{}.{}.epoch.csv".format(model_desc, FLAGS.dataset), index = False)
    labels_df.to_csv("../Results/{}.{}.predictions.csv".format(model_desc, FLAGS.dataset), index = False)

Overwriting train.py


In [16]:
!python3 train.py -epochs 2000 -early_stopping 100 -hidden1 20 -hidden2 10 -dropout 0 -attention_dim 10 -attention_bias 5 -model gcn_cheby -max_degree 3 -dataset test -save_test False -save_validation True

|Training| 6, |Validation| 1, |Testing| 2
False
Calculating Chebyshev polynomials up to order 3...
2019-06-30 22:41:32.372531: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Epoch: 0020 train_loss= 0.01844 train_acc= 1.00000 val_loss= 0.01817 val_acc= 1.00000 time= 0.00826
Epoch: 0040 train_loss= 0.01453 train_acc= 1.00000 val_loss= 0.01438 val_acc= 1.00000 time= 0.00632
Epoch: 0060 train_loss= 0.01181 train_acc= 1.00000 val_loss= 0.01169 val_acc= 1.00000 time= 0.00631
Epoch: 0080 train_loss= 0.00976 train_acc= 1.00000 val_loss= 0.00967 val_acc= 1.00000 time= 0.00387
Epoch: 0100 train_loss= 0.00817 train_acc= 1.00000 val_loss= 0.00810 val_acc= 1.00000 time= 0.00651
/home/samuelstentz/.conda/envs/pyrosetta/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/samuelstentz/.conda/envs/pyrosetta/lib/python3.6/site-packages

Epoch: 1540 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00945
Epoch: 1560 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00522
Epoch: 1580 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00574
Epoch: 1600 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00643
Epoch: 1620 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00437
Epoch: 1640 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00528
Epoch: 1660 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00501
Epoch: 1680 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00305
Epoch: 1700 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00599
Epoch: 1720 train_loss= 0.00002 train_acc= 1.00000 val_loss= 0.00002 val_acc= 1.00000 time= 0.00580


In [2]:
%time !python3 train.py -epochs 2000 -early_stopping 100 -hidden1 20 -hidden2 10 -dropout 0.5 -attention_dim 10 -attention_bias 5 -model gcn -max_degree 3 -dataset selector_k_nearest_ratio_1_params_onehot_distance -save_test False -save_validation True

|Training| 6, |Validation| 1, |Testing| 2
Calculating Chebyshev polynomials up to order 3...
2019-06-28 14:10:27.554643: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Epoch: 0020 train_loss= 0.02935 train_acc= 1.00000 val_loss= 3.39002 val_acc= 0.00000 time= 0.00168
Epoch: 0040 train_loss= 0.00593 train_acc= 1.00000 val_loss= 5.85809 val_acc= 0.00000 time= 0.00160
Epoch: 0060 train_loss= 0.01794 train_acc= 1.00000 val_loss= 6.65260 val_acc= 0.00000 time= 0.00346
Epoch: 0080 train_loss= 0.00538 train_acc= 1.00000 val_loss= 7.18337 val_acc= 0.00000 time= 0.00269
Epoch: 0100 train_loss= 0.00519 train_acc= 1.00000 val_loss= 7.44468 val_acc= 0.00000 time= 0.00282
/home/samuelstentz/.conda/envs/pyrosetta/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/samuelstentz/.conda/envs/pyrosetta/lib/python3.6/site-packages/numpy

Epoch: 1540 train_loss= 0.00038 train_acc= 1.00000 val_loss= 10.81698 val_acc= 0.00000 time= 0.00386
Epoch: 1560 train_loss= 0.00037 train_acc= 1.00000 val_loss= 11.67504 val_acc= 0.00000 time= 0.00304
Epoch: 1580 train_loss= 0.00035 train_acc= 1.00000 val_loss= 11.57590 val_acc= 0.00000 time= 0.00334
Epoch: 1600 train_loss= 0.00038 train_acc= 1.00000 val_loss= 12.67013 val_acc= 0.00000 time= 0.00219
Epoch: 1620 train_loss= 0.00052 train_acc= 1.00000 val_loss= 12.66233 val_acc= 0.00000 time= 0.00232
Epoch: 1640 train_loss= 0.00035 train_acc= 1.00000 val_loss= 12.49781 val_acc= 0.00000 time= 0.00168
Epoch: 1660 train_loss= 0.00032 train_acc= 1.00000 val_loss= 12.07646 val_acc= 0.00000 time= 0.00233
Epoch: 1680 train_loss= 0.00030 train_acc= 1.00000 val_loss= 11.60845 val_acc= 0.00000 time= 0.00247
Epoch: 1700 train_loss= 0.00028 train_acc= 1.00000 val_loss= 11.23469 val_acc= 0.00000 time= 0.00276
Epoch: 1720 train_loss= 0.00026 train_acc= 1.00000 val_loss= 10.79869 val_acc= 0.00000 time

In [1]:
import os
%time !python3 consolidate_results.py

CPU times: user 15.6 ms, sys: 15.6 ms, total: 31.2 ms
Wall time: 5.58 s


[]
